<a href="https://colab.research.google.com/github/ethanlin1126/Data_Mining/blob/main/113_2%E8%B3%87%E6%96%99%E6%8E%A2%E5%8B%98%E4%BD%9C%E6%A5%AD%E4%BA%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

金門縣政府提供教育相關數據

https://drive.google.com/drive/folders/1ZnEmLN_Gbx074-dv_8m3chn2vvhaeUSK?usp=sharing

金門縣康軒國中小使用數據

https://docs.google.com/spreadsheets/d/1LvcLpeVNIa-OfICFnakSbzQgSKBCkfFACmQRIKekgRA/edit?usp=sharing

## 資料匯入與清洗

In [35]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [36]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1LvcLpeVNIa-OfICFnakSbzQgSKBCkfFACmQRIKekgRA/edit?usp=sharing')

In [37]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('anonymized_xapi').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,id,json,timestamp
0,281744,"{""id"": ""5d3ddc37-c43c-4342-8015-c1697ac5148b"",...",6/11/24 22:17
1,281746,"{""id"": ""b27139a8-101a-4d94-9f24-0fb6be429de7"",...",6/11/24 22:17
2,281747,"{""id"": ""73d06d71-56b5-4fd2-a951-7f3e485294ba"",...",6/11/24 22:17
3,281748,"{""id"": ""3c0c29d1-c8f9-441f-a820-bafd90e0fc58"",...",6/11/24 22:17
4,281749,"{""id"": ""6cce33e0-b8d4-47f4-a00d-6dd335c409c2"",...",6/11/24 22:17


In [38]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
alldays = max(set(df['timestamp'].dt.date)) - min(set(df['timestamp'].dt.date))

<ipython-input-38-d2499edfa9a8>:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [39]:
# 總共紀錄的天數
alldays.days

37

In [40]:
# 總共學生行為
len(df)

42928

In [41]:
# 導入json
import json
df_expanded = pd.json_normalize(df['json'].apply(json.loads))

In [42]:
type(df_expanded['timestamp'][0])

str

In [43]:
df_expanded['id'] = df['id']
df_expanded['timestamp'] = pd.to_datetime(df['timestamp'])
df_expanded['Date'] = df_expanded['timestamp'].dt.date    # 提取日期部分
df_expanded['Time'] = df_expanded['timestamp'].dt.time    # 提取時間部分
df_expanded['Weekday'] = df_expanded['timestamp'].dt.day_name()  # 以星期幾的名稱表示（例如：Monday）
df_expanded.head()

,id,timestamp,actor.objectType,actor.name,actor.account.homePage,actor.account.name,verb.id,verb.display.zh-TW,object.objectType,object.id,...,context.team.mailto,context.team.mbox,object.definition.extensions.https://w3id.org/xapi/acrossx/extensions/alignment,object.definition.name.zh-TW,object.definition.interactionType,object.definition.correctResponsesPattern,object.definition.choices,Date,Time,Weekday
0,281744,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
1,281746,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/paused,暫停,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
2,281747,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
3,281748,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/paused,暫停,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
4,281749,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday


In [44]:
import pandas as pd

# 讀取 DataFrame（假設已經有 df）
df['timestamp'] = pd.to_datetime(df['timestamp'])  # 轉換為日期時間格式
df['Date'] = df['timestamp'].dt.date               # 提取日期
df['Time'] = df['timestamp'].dt.time               # 提取時間
df['Weekday'] = df['timestamp'].dt.day_name()      # 提取星期幾
df['Hour'] = df['timestamp'].dt.hour               # 提取小時

# 檢查是否有遺漏值
missing_values = df.isnull().sum()
print(missing_values)


id           0
json         0
timestamp    0
Date         0
Time         0
Weekday      0
Hour         0
dtype: int64


## 分析每位學生的使用頻率

In [52]:
grouped = df_expanded.groupby(['actor.name', 'Date']).agg({
    'timestamp': list,
    'verb.display.zh-TW': list,
    'object.id': list
}).reset_index()

In [53]:
# 將學生每日的使用情況紀錄
grouped['Date'] = pd.to_datetime(grouped['Date'])
grouped['Weekday'] = grouped['Date'].dt.day_name()  # 以星期幾的名稱表示（例如：Monday）
grouped

,actor.name,Date,timestamp,verb.display.zh-TW,object.id,Weekday
0,學生298,2024-06-11,"[2024-06-11 22:17:00, 2024-06-11 22:17:00, 202...","[觀看, 暫停, 觀看, 暫停, 觀看, 暫停, 開始嘗試]",[https://ksjh-km.kschool.com.tw/video-player/4...,Tuesday
1,學生298,2024-06-12,"[2024-06-12 11:39:00, 2024-06-12 11:46:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
2,學生298,2024-06-17,"[2024-06-17 13:39:00, 2024-06-17 13:48:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
3,學生298,2024-06-19,"[2024-06-19 11:16:00, 2024-06-19 11:51:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
4,學生298,2024-06-24,"[2024-06-24 13:47:00, 2024-06-24 14:04:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
...,...,...,...,...,...,...
868,學生590,2024-06-25,"[2024-06-25 13:06:00, 2024-06-25 13:07:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://saps-km.kschool.com.tw/student-quiz-a...,Tuesday
869,學生591,2024-06-25,"[2024-06-25 13:07:00, 2024-06-25 13:09:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://saps-km.kschool.com.tw/student-quiz-a...,Tuesday
870,學生592,2024-06-28,"[2024-06-28 15:09:00, 2024-06-28 15:10:00]","[查看, 查看]",[https://saps-km.kschool.com.tw/teacher-readin...,Friday
871,學生593,2024-07-02,[2024-07-02 11:21:00],[查看],[https://ksjh-km.kschool.com.tw/teacher-readin...,Tuesday


In [54]:
# 學生人數
len(set(grouped['actor.name']))

297

根據每位學生的瀏覽日期，來去畫出不同學生瀏覽影片的天數

In [55]:
# 每位學生瀏覽的日期 並排序由高到低
student_days = grouped.groupby('actor.name')['Date'].nunique()
student_days.colnames = ['actor.name', 'Date']
student_days = student_days.reset_index()
student_days = student_days.sort_values(by='Date', ascending=False)
student_days

,actor.name,Date
17,學生315,12
16,學生314,11
13,學生311,11
2,學生300,11
14,學生312,11
...,...,...
292,學生590,1
293,學生591,1
294,學生592,1
295,學生593,1


In [56]:
# 最常使用學生的使用情形
grouped[grouped['actor.name'] == '學生315']

,actor.name,Date,timestamp,verb.display.zh-TW,object.id,Weekday
164,學生315,2024-06-12,"[2024-06-12 08:40:00, 2024-06-12 08:57:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
165,學生315,2024-06-13,"[2024-06-13 12:25:00, 2024-06-13 12:55:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Thursday
166,學生315,2024-06-14,"[2024-06-14 12:24:00, 2024-06-14 12:36:00, 202...","[開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Friday
167,學生315,2024-06-17,"[2024-06-17 12:27:00, 2024-06-17 12:59:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
168,學生315,2024-06-18,"[2024-06-18 12:28:00, 2024-06-18 12:53:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Tuesday
169,學生315,2024-06-19,"[2024-06-19 07:26:00, 2024-06-19 07:43:00, 202...","[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
170,學生315,2024-06-21,"[2024-06-21 12:30:00, 2024-06-21 13:10:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Friday
171,學生315,2024-06-23,"[2024-06-23 15:47:00, 2024-06-23 15:47:00, 202...","[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Sunday
172,學生315,2024-06-24,"[2024-06-24 12:31:00, 2024-06-24 13:12:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
173,學生315,2024-06-25,"[2024-06-25 12:24:00, 2024-06-25 12:50:00, 202...","[開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Tuesday


In [57]:
grouped[grouped['actor.name'] == '學生314']

,actor.name,Date,timestamp,verb.display.zh-TW,object.id,Weekday
153,學生314,2024-06-12,"[2024-06-12 08:40:00, 2024-06-12 08:57:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
154,學生314,2024-06-13,"[2024-06-13 12:41:00, 2024-06-13 12:43:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Thursday
155,學生314,2024-06-14,"[2024-06-14 12:25:00, 2024-06-14 12:25:00, 202...","[回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Friday
156,學生314,2024-06-17,"[2024-06-17 12:26:00, 2024-06-17 12:46:00, 202...","[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
157,學生314,2024-06-18,"[2024-06-18 12:27:00, 2024-06-18 13:13:00, 202...","[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Tuesday
158,學生314,2024-06-19,"[2024-06-19 07:26:00, 2024-06-19 08:09:00, 202...","[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
159,學生314,2024-06-21,"[2024-06-21 12:35:00, 2024-06-21 13:09:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Friday
160,學生314,2024-06-24,"[2024-06-24 12:26:00, 2024-06-24 12:56:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
161,學生314,2024-06-25,"[2024-06-25 13:09:00, 2024-06-25 13:09:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Tuesday
162,學生314,2024-06-26,"[2024-06-26 12:26:00, 2024-06-26 12:52:00, 202...","[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday


由數據可知，最常使用的學生多數是在回答問題而非觀看影片

In [58]:
# 計算每個 Date 值出現多少人
date_counts = student_days['Date'].value_counts().reset_index()
date_counts.columns = ['Date', 'Count']

# 使用 Plotly 繪製條形圖
fig = px.bar(date_counts.sort_values(by='Date'),
             x='Date', y='Count',
             labels={'Date': '天數', 'Count': '人數'},
             title='學生觀看天數')

# 顯示圖表
fig.show()

由上圖可觀察出，多數學生不常使用平台

## 分析學生觀看影片情形

In [59]:
# 提取影片的類型、標題和主題
video_activity_info = df_expanded[['actor.name', 'Date', 'Time',	'Weekday', 'verb.display.zh-TW']]

# 添加影片的定義類型和主題
video_activity_info['theme'] = df_expanded['object.definition.extensions.https://w3id.org/xapi/acrossx/extensions/alignment'].apply(lambda x: x[0] if isinstance(x, list) and x else None)
video_activity_info

<ipython-input-59-f45f0977ad26>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,actor.name,Date,Time,Weekday,verb.display.zh-TW,theme
0,學生298,2024-06-11,22:17:00,Tuesday,觀看,None
1,學生298,2024-06-11,22:17:00,Tuesday,暫停,None
2,學生298,2024-06-11,22:17:00,Tuesday,觀看,None
3,學生298,2024-06-11,22:17:00,Tuesday,暫停,None
4,學生298,2024-06-11,22:17:00,Tuesday,觀看,None
...,...,...,...,...,...,...
42923,學生584,2024-07-18,11:24:00,Thursday,暫停,1-2物質進出細胞的方式
42924,學生584,2024-07-18,11:26:00,Thursday,觀看,1-2物質進出細胞的方式
42925,學生584,2024-07-18,11:26:00,Thursday,暫停,1-2物質進出細胞的方式
42926,學生584,2024-07-18,11:28:00,Thursday,觀看,1-2物質進出細胞的方式


In [60]:
# 學生執行功能
distinct_types = video_activity_info['verb.display.zh-TW'].unique()
distinct_types

array(['觀看', '暫停', '開始嘗試', '回答', '完成', '查看'], dtype=object)

In [61]:
# 所有影片標題
theme = video_activity_info['theme'].unique()
theme

array([None, '第4單元 複習影片', '第3單元 複習影片', 'MA ,32,', 'NA ,22,', 'L11重點句型',
       'SO ,12,', 'L6戰後臺灣的文化與社會發展', 'SO ,52,', 'L6 Word Bank', 'CH ,32,',
       'L6戰後臺灣的經濟變遷', 'MA ,12,', 'L12文意理解', 'NA ,32,',
       'B4L6-2_文法動畫_if_although句型', 'B4L6-1_文法動畫_不定代名詞',
       'B4L6 Reading 動畫', 'B4L6 Dialogue 動畫', 'EN ,5,', '複習&統整CH4',
       '複習&統整CH5', 'L12重點句型', 'L11文意理解', 'CH ,12,', 'L6臺灣的區域差異',
       'L6臺灣的區域特色', 'L6臺灣六都的特色', 'L8 文意理解', 'L6社會安全與國家責任', 'L8 詞成語',
       'L8 形音義', 'NA ,12,', '4-2反推不等式', '4-2應用題--幾何', '3-6鳥類和哺乳類',
       '3-6魚類、兩生類和爬蟲類', '3-6節肢動物門', '3-6軟體、環節、刺絲胞、棘皮、扁形動物門', '軍犬立大功',
       '和喜鵲學築巢', 'CH ,51,52,', '迎向第一次段考讀書計畫表', '國語二上第7課教學動畫愛漂亮的國王',
       '1-1生命現象', '1-1細胞的發現、型態與功能', '1-1動、植物細胞構造', '1-2組成細胞的物質',
       '1-2物質進出細胞的方式'], dtype=object)

In [62]:
# 學生每日行為
# Filter out rows where 'theme' is None for meaningful pairing
filtered_df = video_activity_info.dropna(subset=['theme'])

# Pivot the table to show 'verb.display.zh-TW' as columns and 'theme' as corresponding values
pivoted_df = filtered_df.pivot_table(index=['actor.name', 'Date'], columns='verb.display.zh-TW', values='theme', aggfunc=lambda x: ', '.join(x))

# Reset index for better readability
pivoted_df = pivoted_df.reset_index()
pivoted_df

verb.display.zh-TW,actor.name,Date,回答,完成,暫停,查看,觀看,開始嘗試
0,學生298,2024-06-24,"SO ,12,, SO ,12,, SO ,12,, SO ,12,, SO ,12,, S...","SO ,12,",NaN,NaN,NaN,"SO ,12,"
1,學生299,2024-06-26,NaN,NaN,NaN,NaN,NaN,"NA ,12,, NA ,12,"
2,學生299,2024-06-27,NaN,NaN,NaN,NaN,NaN,"SO ,12,"
3,學生300,2024-06-24,"CH ,12,, CH ,12,, CH ,12,, CH ,12,, CH ,12,, C...","CH ,12,, SO ,12,, SO ,12,",NaN,NaN,NaN,"CH ,12,, SO ,12,, SO ,12,"
4,學生300,2024-06-25,"NA ,12,, NA ,12,, NA ,12,, NA ,12,, NA ,12,, N...","NA ,12,",NaN,NaN,NaN,"NA ,12,, NA ,12,, NA ,12,, NA ,12,, NA ,12,"
...,...,...,...,...,...,...,...,...
233,學生590,2024-06-25,"EN ,5,, EN ,5,, EN ,5,, EN ,5,, EN ,5,, EN ,5,...","EN ,5,, EN ,5,, EN ,5,",NaN,NaN,NaN,"EN ,5,, EN ,5,, EN ,5,"
234,學生591,2024-06-25,"EN ,5,, EN ,5,, EN ,5,, EN ,5,, EN ,5,, EN ,5,...","EN ,5,, EN ,5,, EN ,5,",NaN,NaN,NaN,"EN ,5,, EN ,5,, EN ,5,, EN ,5,"
235,學生592,2024-06-28,NaN,NaN,NaN,"軍犬立大功, 和喜鵲學築巢",NaN,NaN
236,學生593,2024-07-02,NaN,NaN,NaN,迎向第一次段考讀書計畫表,NaN,NaN


In [63]:
# 學生在不同單元的情形
# Assuming the dataframe is already loaded into df and contains columns like 'actor.name', 'Date', 'verb.display.zh-TW', 'theme'

# Filter out rows where theme is None (if applicable)
df_filtered = video_activity_info.dropna(subset=['theme'])

# Group by 'actor.name', 'theme', and 'verb.display.zh-TW' (behavior)
behavior_counts = df_filtered.groupby(['actor.name', 'Date', 'theme', 'verb.display.zh-TW']).size().unstack(fill_value=0)

# Display the behavior counts per student and theme
behavior_counts

verb.display.zh-TW                       回答  完成  暫停  查看  觀看  開始嘗試
actor.name Date       theme                                      
學生298      2024-06-24 SO ,12,            40   1   0   0   0     1
學生299      2024-06-26 NA ,12,             0   0   0   0   0     2
           2024-06-27 SO ,12,             0   0   0   0   0     1
學生300      2024-06-24 CH ,12,            40   1   0   0   0     1
                      SO ,12,            80   2   0   0   0     2
...                                      ..  ..  ..  ..  ..   ...
學生591      2024-06-25 EN ,5,             30   3   0   0   0     4
學生592      2024-06-28 和喜鵲學築巢              0   0   0   1   0     0
                      軍犬立大功               0   0   0   1   0     0
學生593      2024-07-02 迎向第一次段考讀書計畫表        0   0   0   1   0     0
學生594      2024-07-02 國語二上第7課教學動畫愛漂亮的國王   0   0   1   0   1     0

[357 rows x 6 columns]

In [64]:
behavior_counts.index

MultiIndex([('學生298', 2024-06-24,           'SO ,12,'),
            ('學生299', 2024-06-26,           'NA ,12,'),
            ('學生299', 2024-06-27,           'SO ,12,'),
            ('學生300', 2024-06-24,           'CH ,12,'),
            ('學生300', 2024-06-24,           'SO ,12,'),
            ('學生300', 2024-06-25,           'NA ,12,'),
            ('學生300', 2024-06-26,           'NA ,12,'),
            ('學生300', 2024-06-27,           'SO ,12,'),
            ('學生301', 2024-06-24,           'CH ,12,'),
            ('學生301', 2024-06-26,           'NA ,12,'),
            ...
            ('學生589', 2024-06-25,         'L6臺灣的區域特色'),
            ('學生589', 2024-06-25,          '複習&統整CH4'),
            ('學生589', 2024-06-25,          '複習&統整CH5'),
            ('學生589', 2024-06-26,          '複習&統整CH5'),
            ('學生590', 2024-06-25,            'EN ,5,'),
            ('學生591', 2024-06-25,            'EN ,5,'),
            ('學生592', 2024-06-28,            '和喜鵲學築巢'),
            ('學生592', 2024-06-28

In [65]:
# 加入index
from datetime import date
# Accessing 'actor.name' and 'theme' from MultiIndex
actor_names = behavior_counts.index.get_level_values('actor.name')
themes = behavior_counts.index.get_level_values('theme')
dates = behavior_counts.index.get_level_values('Date')

# Example: Combine into a DataFrame for easy viewing
df_with_index = pd.DataFrame({
    'actor.name': actor_names,
    'theme': themes,
    'Date': dates
})
df_with_index

,actor.name,theme,Date
0,學生298,"SO ,12,",2024-06-24
1,學生299,"NA ,12,",2024-06-26
2,學生299,"SO ,12,",2024-06-27
3,學生300,"CH ,12,",2024-06-24
4,學生300,"SO ,12,",2024-06-24
...,...,...,...
352,學生591,"EN ,5,",2024-06-25
353,學生592,和喜鵲學築巢,2024-06-28
354,學生592,軍犬立大功,2024-06-28
355,學生593,迎向第一次段考讀書計畫表,2024-07-02


In [66]:
# 合併有index的資料和次數資料
merged_df = pd.merge(df_with_index, behavior_counts, on=['actor.name', 'theme', 'Date'])
merged_df

,actor.name,theme,Date,回答,完成,暫停,查看,觀看,開始嘗試
0,學生298,"SO ,12,",2024-06-24,40,1,0,0,0,1
1,學生299,"NA ,12,",2024-06-26,0,0,0,0,0,2
2,學生299,"SO ,12,",2024-06-27,0,0,0,0,0,1
3,學生300,"CH ,12,",2024-06-24,40,1,0,0,0,1
4,學生300,"SO ,12,",2024-06-24,80,2,0,0,0,2
...,...,...,...,...,...,...,...,...,...
352,學生591,"EN ,5,",2024-06-25,30,3,0,0,0,4
353,學生592,和喜鵲學築巢,2024-06-28,0,0,0,1,0,0
354,學生592,軍犬立大功,2024-06-28,0,0,0,1,0,0
355,學生593,迎向第一次段考讀書計畫表,2024-07-02,0,0,0,1,0,0


In [78]:
view = merged_df[merged_df['觀看'] >= 0]

In [79]:
# 影片觀看情形
import plotly.express as px

# Convert 'Date' to datetime
view['Date'] = pd.to_datetime(view['Date'])

# Group the data by Date and theme, summing the '觀看' counts
df_view_counts = view.groupby(['Date', 'theme'])['觀看'].sum().reset_index()

# Plot using Plotly
fig = px.bar(df_view_counts, x='Date', y='觀看', color='theme',
             title="Number of Views by Theme Over Time",
             labels={'觀看': 'View Count', 'theme': 'Theme'},
             barmode='stack')

# Update layout for better visualization
fig.update_layout(xaxis_title="Date", yaxis_title="View Count",
                  legend_title="Theme", title_x=0.5)

# Show the plot
fig.show()

分析科目使用比例

In [80]:
import pandas as pd
import plotly.express as px

# 定義科目分類
subject_mapping = {
    # 英文科
    'L11重點句型': '英文科', 'L12文意理解': '英文科', 'B4L6-2_文法動畫_if_although句型': '英文科',
    'B4L6-1_文法動畫_不定代名詞': '英文科', 'B4L6 Reading 動畫': '英文科', 'B4L6 Dialogue 動畫': '英文科',
    'L12重點句型': '英文科', 'L11文意理解': '英文科', 'L8 文意理解': '英文科',

    # 社會科
    'L6戰後臺灣的文化與社會發展': '社會科', 'L6臺灣的區域差異': '社會科',
    'L6臺灣的區域特色': '社會科', 'L6臺灣六都的特色': '社會科', 'L6社會安全與國家責任': '社會科',

    # 自然科
    '1-1生命現象': '自然科', '1-1細胞的發現、型態與功能': '自然科', '1-1動、植物細胞構造': '自然科',
    '1-2組成細胞的物質': '自然科', '1-2物質進出細胞的方式': '自然科',
    '3-6魚類、兩生類和爬蟲類': '自然科', '3-6節肢動物門': '自然科', '3-6軟體、環節、刺絲胞、棘皮、扁形動物門': '自然科',
    '3-6鳥類和哺乳類': '自然科',

    # 數學科
    '4-2反推不等式': '數學科', '4-2應用題--幾何': '數學科',

    # 國文科
    '軍犬立大功': '國文科', '和喜鵲學築巢': '國文科', '迎向第一次段考讀書計畫表': '國文科',
    '國語二上第7課教學動畫愛漂亮的國王': '國文科'
}

# 確保 `view` DataFrame 存在
if 'view' not in locals():
    raise ValueError("請確保 `view` DataFrame 已正確載入！")

# 確保 'Date' 欄位存在並轉換為 datetime 格式
if 'Date' in view.columns:
    view['Date'] = pd.to_datetime(view['Date'])
else:
    raise KeyError("`view` DataFrame 缺少 'Date' 欄位！")

# 依據 `theme` 欄位對應 `subject_mapping`
view['subject'] = view['theme'].map(subject_mapping)
filtered_view = view.dropna(subset=['subject'])  # 移除未分類的行

# **確保所有科目都有資料，即使是 0**
all_subjects = ['英文科', '社會科', '自然科', '數學科', '國文科']
df_subject_views = filtered_view.groupby('subject', as_index=False)['觀看'].sum()

# 確保所有科目都出現在圖表上
df_subject_views = df_subject_views.set_index('subject').reindex(all_subjects, fill_value=0).reset_index()

# 繪製長條圖
fig = px.bar(df_subject_views, x='subject', y='觀看',
             title="Number of Views: Different Subjects",
             labels={'subject': 'Subject', '觀看': 'View Count'},
             color='subject')

# 更新視覺效果
fig.update_layout(xaxis_title="Subject", yaxis_title="Total Views",
                  title_x=0.5, showlegend=False)

# 顯示圖表
fig.show()


得知英文、社會及自然科的老師會使用平台教學，且自然科使用最頻繁<br>
國文和數學老師不太使用平台教學